In [ ]:
# Misc
import random
import numpy as np
import pandas as pd
import tqdm
from tqdm import tqdm
from tqdm import tqdm_notebook
import math
import os
import time
import sys

In [ ]:
# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Sklearn
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

In [ ]:
# Pytorch
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
# Keras /
from tensorflow import keras
from keras.utils.np_utils import to_categorical

In [ ]:
# # Helpers
# from Adverse import lowProFool, deepfool
from Metrics import *
from Adverse import deepfool

In [ ]:
import copy

#### setting and confirm the test data

In [ ]:
test_seed = 98
num = 120
num_seed = 175

In [ ]:
outfile = 'lpf'+str(test_seed)+'_'+str(num)+'_'+str(num_seed)+'.csv'
outfile

In [ ]:
data = pd.read_csv('path + total_data.csv')
# data

In [ ]:
ori_data = copy.deepcopy(data)
ori_y = ori_data['isFake']
ori_X = ori_data.drop(['isFake'],axis=1)
ori_X_train, ori_X_test, ori_y_train, ori_y_test=train_test_split(ori_X,ori_y,test_size=0.2,random_state=test_seed,stratify=ori_y)

In [ ]:
s1 = set(ori_X_test.index)
s1 = np.array(list(s1))
s = s1[s1<200]
location = []
loc = list(s*8)
for i in range(len(loc)):
    for j in range(8):
        location.append(loc[i]+j)
len(location)

In [ ]:
# np.array(set(location))

In [ ]:
len(ori_X_test)
len(ori_X_train)

In [ ]:
len(s)

#### sample target to attack and creat a filter --------------------------------------------------------------------------------

In [ ]:
random.seed(num_seed)
fea = random.sample(location,num)

In [ ]:
# np.array(set(fea))

In [ ]:
att_fea_loc = []
row_index = []
for i in range(len(fea)):
    att_fea_loc.append([fea[i]//8, fea[i]%8])
    row_index.append(fea[i]//8)
row_index = set(row_index)
len(row_index)

In [ ]:
fil = np.zeros((200, 8))
fil

In [ ]:
for i in range(len(att_fea_loc)):
    fil[att_fea_loc[i][0],att_fea_loc[i][1]] = 1.0
# print(i)

In [ ]:
import copy
data_X = copy.deepcopy(data)
data_X = data_X.drop(['isFake'],axis=1)
# data_X

In [ ]:
import warnings
warnings.filterwarnings('ignore',category=Warning)

In [ ]:
max_data = list(np.max(data_X))
min_data = list(np.min(data_X))
# print(max_data)
# print(min_data)

In [ ]:
df = data
target = 'isFake'
feature_names = list(data.columns)
feature_names = feature_names[0:8]
feature_names

In [ ]:
SEED = 0

In [ ]:
def get_bounds(df):
    low_bounds = df.min().values
    up_bounds = df.max().values
    
    #removing target WARNING ASSUMES TARGET IS LAST
    low_bounds = low_bounds[0:-1]
    up_bounds = up_bounds[0:-1]
    
    return [low_bounds, up_bounds]

# Compute the bounds for clipping
bounds = get_bounds(df)
bounds

In [ ]:
import warnings
warnings.filterwarnings('ignore',category=Warning)

In [ ]:
def normalize(df, target, feature_names, bounds):
    df_return = df.copy()
    
    # Makes sure target does not need scaling
    targets = np.unique(df[target].values)
    #assert(len(targets == 2) and 0 in targets and 1 in targets)
    
    scaler = MinMaxScaler()
    X = df_return[feature_names]
    scaler.fit(X)    
    df_return[feature_names] = scaler.transform(X)
    
    lower_bounds = scaler.transform([bounds[0]])
    upper_bounds = scaler.transform([bounds[1]])

    return scaler, df_return, (lower_bounds[0], upper_bounds[0])

# Normalize the data
scaler, df, bounds = normalize(df, target, feature_names, bounds)

In [ ]:
temp = df.loc[row_index]

In [ ]:
def get_weights(df, target, show_heatmap=False):
    def heatmap(cor):
        plt.figure(figsize=(8,6))
        sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
        plt.show()

    cor = df.corr()
    cor_target = abs(cor[target])

    weights = cor_target[:-1] #removing target WARNING ASSUMES TARGET IS LAST
    weights = weights / np.linalg.norm(weights)

    if show_heatmap:
        heatmap(cor)
            
    return weights.values

# Compute the weihts modelizing the expert's knowledge
weights = get_weights(df, target)
weights

In [ ]:
def split_train_test_valid(df):
    # Train test splits (1:4)
    # Test valid split (1:4)
    df_train, df_test = train_test_split(df, test_size = 0.2, shuffle=True, random_state=SEED,stratify=df['isFake'])
    df_test, df_valid = train_test_split(df_test, test_size = 0.2, shuffle=True, random_state=SEED)
    
    return df_train, df_test

# Split df into train/test/valid
df_train, df_test = split_train_test_valid(df)

In [ ]:
print(len(df_train))
print(len(df_test))
print(len(df_train)+len(df_test))

In [ ]:
print('df_train is fake : ',len(df_train[df_train['isFake']==1]))
print('df_train is real : ',len(df_train[df_train['isFake']==0]))
print('df_test is fake : ',len(df_test[df_test['isFake']==1]))
print('df_test is real : ',len(df_test[df_test['isFake']==0]))

In [ ]:
# feature importance
nu_1 = np.array([1., 0., 1., 1., 0., 0., 0., 1.]) # 只改前四低相關的特徵(1,3,4,8)
nu_2 = np.array([0., 1., 0., 0., 1., 1., 1., 0.]) # 只改前四高相關的特徵(2,5,6,7)

nu_3 = np.array([1., 1., 1., 0., 0., 0., 0., 0.])

nu_4 = np.array([1., 1., 0., 0., 0., 0., 0., 0.])

In [ ]:
config = {'Dataset'     : 'twitter',
         'MaxIters'     : 10000,
         'Alpha'        : 0.001,
         'Lambda'       : 8.5,
         'TrainData'    : df_train,
         'TestData'     : df_test,
         'Scaler'       : scaler,
         'FeatureNames' : feature_names,
         'Target'       : target,
         'Weights'      : weights,
         'Bounds'       : bounds}

In [ ]:
def get_model(conf, load=False):
    
    class GermanNet(nn.Module):
        def __init__(self, D_in, H, D_out):
            super(GermanNet, self).__init__()
            self.linear1 = torch.nn.Linear(D_in, H)
            self.linear2 = torch.nn.Linear(H, H)
            self.linear3 = torch.nn.Linear(H, D_out)
            self.relu = torch.nn.ReLU()
            self.softmax = torch.nn.Softmax(dim=0)

        def forward(self, x):
            h1 = self.relu(self.linear1(x))
            h2 = self.relu(self.linear2(h1))
            h3 = self.relu(self.linear2(h2))
            h4 = self.relu(self.linear2(h3))
            h5 = self.relu(self.linear2(h4))
            h6 = self.relu(self.linear2(h5))
            a3 = self.linear3(h6)
            y = self.softmax(a3)
            return y

    def train(model, criterion, optimizer, X, y, N, n_classes):
        model.train()

        current_loss = 0
        current_correct = 0


        # Training in batches
        for ind in range(0, X.size(0), N):
            indices = range(ind, min(ind + N, X.size(0)) - 1) 
            inputs, labels = X[indices], y[indices]
            inputs = Variable(inputs, requires_grad=True)


            optimizer.zero_grad()

            output = model(inputs)
            _, indices = torch.max(output, 1) # argmax of output [[0.61, 0.12]] -> [0]
            # [[0, 1, 1, 0, 1, 0, 0]] -> [[1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0]]
            preds = torch.tensor(keras.utils.to_categorical(indices, num_classes=n_classes))

            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            current_loss += loss.item()
            current_correct += (preds.int() == labels.int()).sum() /n_classes


        current_loss = current_loss / X.size(0)
        current_correct = current_correct.double() / X.size(0)    

        return preds, current_loss, current_correct.item()
    
    df = conf['TrainData']
    target = conf['Target']
    feature_names = conf['FeatureNames']
                        
    n_classes = len(np.unique(df[target]))
    X_train = torch.FloatTensor(df[feature_names].values)
    y_train = keras.utils.to_categorical(df[target], n_classes)
    y_train = torch.FloatTensor(y_train)

    D_in = X_train.size(1)
    D_out = y_train.size(1)

    epochs = 6000
    batch_size = 100
    H = 100
    net = GermanNet(D_in, H, D_out)

    lr = 1e-4    
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    for epoch in range(epochs):
        preds, epoch_loss, epoch_acc = train(net, criterion, optimizer, X_train, y_train, batch_size, n_classes)     
        if (epoch % 1000 == 0):
            print("> epoch {:.0f}\tLoss {:.5f}\tAcc {:.5f}".format(epoch, epoch_loss, epoch_acc))

    net.eval()
    
    return net

# Train neural network
model = get_model(config)
config['Model'] = model

In [ ]:
# Compute accuracy on test set
y_true = df_test[target]
x_test = torch.FloatTensor(df_test[feature_names].values)
y_pred = model(x_test)
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
print("Accuracy score on test data", accuracy_score(y_true, y_pred))
print("Recall score on test data", recall_score(y_true, y_pred))
print("F1 score on test data", f1_score(y_true, y_pred))
print("confusion_matrix", confusion_matrix(y_true, y_pred))


In [ ]:
config['TestData'] = temp
# config['TestData']

In [ ]:
# config['TestData'] = df[df['isFake']==1].sample(n=75, random_state = 6)
# config['TestData']

#### LowProFool

In [ ]:
def zero_gradients(x):
    if isinstance(x, torch.Tensor):
        if x.grad is not None:
            x.grad.detach_()
            x.grad.zero_()
    elif isinstance(x, collections.abc.Iterable):
        for elem in x:
            zero_gradients(elem)
            
# Clipping function
def clip(current, low_bound, up_bound):
    assert(len(current) == len(up_bound) and len(low_bound) == len(up_bound))
    low_bound = torch.FloatTensor(low_bound)
    #print('low_bound',low_bound)
    up_bound = torch.FloatTensor(up_bound)
    clipped = torch.max(torch.min(current, up_bound), low_bound)
    #print(current)
    #print('clipped',clipped)
    return clipped


In [ ]:
def lowProFool(x, model, weights, bounds, maxiters, alpha, lambda_, fil):
    """
    Generates an adversarial examples x' from an original sample x

    :param x: tabular sample
    :param model: neural network
    :param weights: feature importance vector associated with the dataset at hand
    :param bounds: bounds of the datasets with respect to each feature
    :param maxiters: maximum number of iterations ran to generate the adversarial examples
    :param alpha: scaling factor used to control the growth of the perturbation
    :param lambda_: trade off factor between fooling the classifier and generating imperceptible adversarial example
    :return: original label prediction, final label prediction, adversarial examples x', iteration at which the class changed
    """
#     print(x)
#     r = Variable(torch.FloatTensor(1e-4 * np.ones(x.numpy().shape)), requires_grad=True)
    #nu_1 = np.array([1., 0., 1., 1., 0., 0., 0., 1.]) # 只改前四低相關的特徵(1,3,4,8)
    #nu_2 = np.array([0., 1., 0., 0., 1., 1., 1., 0.]) # 只改前四高相關的特徵(2,5,6,7)
#     nu_3 = np.array([1., 1., 1., 0., 0., 0., 0., 0.]) # 前四高相關的特徵(2,5,6,7)
#     nu_4 = np.array([1., 1., 0., 0., 0., 0., 0., 0.])

    r = Variable(torch.FloatTensor(1e-4 * fil), requires_grad=True)
#     print(r)
    v = torch.FloatTensor(np.array(weights))
    #print(v)
    output = model.forward(x + r)
    orig_pred = output.max(0, keepdim=True)[1].cpu().numpy()
    #print(orig_pred)
    target_pred = np.abs(1 - orig_pred)
    #print(target_pred)
    
    target = [0., 1.] if target_pred == 1 else [1., 0.]
    target = Variable(torch.tensor(target, requires_grad=False)) 
    
    lambda_ = torch.tensor([lambda_])
    
    bce = nn.BCELoss()
    l1 = lambda v, r: torch.sum(torch.abs(v * r)) #L1 norm
    l2 = lambda v, r: torch.sqrt(torch.sum(torch.mul(v * r,v * r))) #L2 norm
    
    best_norm_weighted = np.inf
    best_pert_x = x
    
    loop_i, loop_change_class = 0, 0
#     while loop_i < 3:
    while loop_i < maxiters:
        #print('r',r)    
        zero_gradients(r)
        
        # Computing loss 
        loss_1 = bce(output, target)
        loss_2 = l2(v, r)
        loss = (loss_1 + lambda_ * loss_2)

        # Get the gradient
        loss.backward(retain_graph=True)
        grad_r = r.grad.data.cpu().numpy().copy()
        #print(grad_r)
        
        # Guide perturbation to the negative of the gradient
        ri = - grad_r
    
        # limit huge step
        ri *= alpha

        # Adds new perturbation to total perturbation
        #r = r.clone().detach().cpu().numpy() + ri
        
        temp = []
        for i in range(len(r)):
            if r[i].tolist() == 0:
                temp.append(0.0)
                #temp.append(r.clone().detach().cpu().numpy()[i])
            else:
                temp.append(r.clone().detach().cpu().numpy()[i] + ri[i])
        r = np.array(temp)
        #print('r1',r)
        
        
        # For later computation
        r_norm_weighted = np.sum(np.abs(r * weights))
        
        # Ready to feed the model
        r = Variable(torch.FloatTensor(r), requires_grad=True) 
        #print('r2',r)
        
        # Compute adversarial example
        #print('x',x)
        #print('r',r)
        xprime = x + r
#         if loop_i == 19999:

#         print('xprime',xprime)
        
        # Clip to stay in legitimate bounds
        xprime = clip(xprime, bounds[0], bounds[1])
#         if loop_i == 19999:
#         print('xprime',xprime)
        
        
#         
        
        # Classify adversarial example
        output = model.forward(xprime)
        output_pred = output.max(0, keepdim=True)[1].cpu().numpy()
        
        # Keep the best adverse at each iterations
        if output_pred != orig_pred and r_norm_weighted < best_norm_weighted:
            best_norm_weighted = r_norm_weighted
            best_pert_x = xprime

        if output_pred == orig_pred:
            loop_change_class += 1
            
#            
#         print('xprime',xprime)
        
        loop_i += 1 
        
        
    # Clip at the end no matter what
    #print(best_pert_x)
    best_pert_x = clip(best_pert_x, bounds[0], bounds[1])
    output = model.forward(best_pert_x)
    #print('output',output)
    output_pred = output.max(0, keepdim=True)[1].cpu().numpy()
    #print('output_pred',output_pred)
    
#     print(best_pert_x)
#     x_temp = []
#     for f in range(10):
#         if fil[f] == 0.0:
#             x_temp.append(x[f])
#             print(x[f])
#             print(best_pert_x[f])
#         else:
#             x_temp.append(best_pert_x[f])
#     best_pert_x = torch.FloatTensor(x_temp) 

    return orig_pred, output_pred, best_pert_x.clone().detach().cpu().numpy(), loop_change_class 

#### single data to obverse

In [ ]:
# c = config['TestData'].to_numpy()[0][0:10]
# c

In [ ]:
# x_tensor = torch.FloatTensor(c)
# x_tensor

In [ ]:
# config['Bounds']
# fil[0][3] == 0.0

In [ ]:
# orig_pred, adv_pred, x_adv, loop_i = lowProFool(x_tensor, config['Model'], config['Weights'], config['Bounds'], config['MaxIters'], config['Alpha'], config['Lambda'],fil[0])

In [ ]:
# c

In [ ]:
# x_adv

In [ ]:
# lowProFool(x_tensor, config['Model'], config['Weights'], config['Bounds'], config['MaxIters'], config['Alpha'], config['Lambda'],fil[1])

In [ ]:
# print(list(data.iloc[0]))
# print(fil[0])

In [ ]:
# max_data_np = np.asarray(max_data)
# min_data_np = np.asarray(min_data)
# v =  x_adv.tolist()*(max_data_np-min_data_np)+min_data_np
# adv_x = np.asarray([round(i) for i in v])
# print(adv_x)
# ori_x = data.iloc[0].tolist()[:-1]
# print(ori_x)

# new_adv_x = []
# for i in range(10):
#     if fil[0][i] == 0:
#         new_adv_x.append(ori_x[i])
#     else:
#         new_adv_x.append(adv_x[i])
# print(new_adv_x)

In [ ]:
# max_data

In [ ]:
# config['TestData'].index

#### lowprofool attack method

In [ ]:
def gen_adv(config, method):
    df_test = config['TestData']
    extra_cols = ['orig_pred', 'adv_pred', 'iters']    
    model = config['Model']
    weights = config['Weights']
    bounds = config['Bounds']
    maxiters = config['MaxIters']
    alpha = config['Alpha']
    lambda_ = config['Lambda']
    #nu = config['nu']
    
    results = np.zeros((len(df_test), len(feature_names) + len(extra_cols)))    
            
    i = -1
    for _, row in tqdm_notebook(df_test.iterrows(), total=df_test.shape[0], desc="{}".format(method)):
        i += 1
        
        x_tensor = torch.FloatTensor(row[config['FeatureNames']])   
        #print(x_tensor)
        #print(row.name)
        #print(fil[row.name])
        if method == 'LowProFool':
            orig_pred, adv_pred, x_adv, loop_i = lowProFool(x_tensor, model, weights, bounds,
                                                             maxiters, alpha, lambda_,fil[row.name])
        elif method == 'Deepfool':
            orig_pred, adv_pred, x_adv, loop_i = deepfool(x_tensor, model, maxiters, alpha,
                                                          bounds, weights=[])
        else:
            raise Exception("Invalid method", method)
            
        results[i] = np.concatenate((x_adv, [orig_pred, adv_pred, loop_i]), axis=0)
        #results[i] = adv_pred
        
        x = pd.DataFrame(results, index=df_test.index, columns = feature_names + extra_cols)
  
    return x

In [ ]:
import warnings
warnings.filterwarnings('ignore',category=Warning)

In [ ]:
adv_sample = gen_adv(config, 'LowProFool')

#### get the adversarial sample and output the dataset with adv.sample

In [ ]:
adv_sample_np = adv_sample[feature_names].to_numpy()
max_data_np = np.asarray(max_data)
min_data_np = np.asarray(min_data)
original_np = []
for i in range(len(adv_sample_np)):
    v = adv_sample_np[i]*(max_data_np-min_data_np)+min_data_np
    v = np.asarray([round(i) for i in v])
    original_np.append(v)
original = pd.DataFrame(original_np)
original_f = pd.DataFrame(original_np)
original['orig_pred'] = list(adv_sample['orig_pred'])
original['adv_pred'] = list(adv_sample['adv_pred'])
original.index = adv_sample.index
original_f.index = adv_sample.index
original_f.columns = feature_names
original_f['isFake'] = [1 for i in range(len(row_index))]
original_f

In [ ]:
original_2 = copy.deepcopy(original)
original_2 = original_2.drop('orig_pred',axis=1)
original_2 = original_2.drop('adv_pred',axis=1)
original_2.columns = feature_names
original_2['isFake'] = [1 for i in range(len(row_index))]

In [ ]:
com_1 = data.iloc[list(set(list(data.index))-set(list(original_2.index)))]

In [ ]:
export = pd.concat((original_2,com_1))

In [ ]:
export = export.sort_index(axis=0)

In [ ]:
export.to_csv(outfile)